# Metis Data Science Bootcamp
## Project 4: Natural Language Processing and Unsupervised Learning



In [1]:
import sys
import re
import os.path
import requests
import time
import pandas as pd

from bs4 import BeautifulSoup as soup
import urllib
from urllib.request import urlopen
import json

from os import path

from pymongo import MongoClient
import requests

# Import required modules.
from azure.cognitiveservices.search.websearch import WebSearchClient
from azure.cognitiveservices.search.websearch.models import SafeSearch
from msrest.authentication import CognitiveServicesCredentials
from azure.cognitiveservices.search.newssearch import NewsSearchClient

from pymongo import MongoClient
import requests



In [2]:
# News site definitions
div_search = {'www.nytimes.com' : { 'head_tag' : ('article', 'id', 'story'), 'text_tag': ('p', 'class', 'css-exrw3m')},
              'www.seattletimes.com' : {'head_tag' : ('div', 'id', 'article-content') ,'text_tag':('p', '', '')},
              'www.washingtonexaminer.com' : {'head_tag' : ('', '', '') ,'text_tag':('p', '', '')},
              'www.msn.com' : {'head_tag' : ('div', 'id', 'maincontent') ,'text_tag':('p', '', '')},
              'www.usatoday.com' : {'head_tag' : ('article', 'class', 'gnt_pr') ,'text_tag':('p', 'class', 'gnt_ar_b_p')},
              'www.politico.com' : {'head_tag' : ('div', 'class', 'story-text') ,'text_tag':('p', 'class', 'story-text__paragraph')},
              'www.mercurynews.com' : {'head_tag' : ('div', 'class', 'body-copy') ,'text_tag':('p', '', '')},
              'www.wsj.com' : {'head_tag' : ('', '', '') ,'text_tag':('p', '', '')},
              'www.newsweek.com' : {'head_tag' : ('', '', '') ,'text_tag':('p', '', '')},
              'www.businessinsider.com' : {'head_tag' : ('', '', '') ,'text_tag':('p', '', '')},
              'www.washingtonpost.com' : {'head_tag' : ('', '', '') ,'text_tag':('p', '', '')},
              'www.chicagotribune.com' : {'head_tag' : ('', '', '') ,'text_tag':('p', '', '')},
              'www.newyorker.com' : {'head_tag' : ('', '', '') ,'text_tag':('p', '', '')},
              'www.lasvegassun.com' : {'head_tag' : ('', '', '') ,'text_tag':('p', '', '')},
              'www.sfchronicle.com' : {'head_tag' : ('', '', '') ,'text_tag':('p', '', '')},
              'www.latimes.com' : {'head_tag' : ('', '', '') ,'text_tag':('p', '', '')},
              'www.nydailynews.com' : {'head_tag' : ('', '', '') ,'text_tag':('p', '', '')},
              'www.washingtontimes.com' : {'head_tag' : ('', '', '') ,'text_tag':('p', '', '')},
              'www.thenation.com' : {'head_tag' : ('', '', '') ,'text_tag':('p', '', '')},
              'www.theguardian.com' : {'head_tag' : ('', '', '') ,'text_tag':('p', '', '')},
              'www.foxnews.com' : {'head_tag' : ('', '', '') ,'text_tag':('p', '', '')},
              'www.theatlantic.com' : {'head_tag' : ('', '', '') ,'text_tag':('p', '', '')},
              'www.sfexaminer.com' : {'head_tag' : ('', '', '') ,'text_tag':('p', '', '')},
              'www.bostonherald.com' : {'head_tag' : ('', '', '') ,'text_tag':('p', '', '')},
              'www.thedailybeast.com' : {'head_tag' : ('', '', '') ,'text_tag':('p', '', '')},
              'www.cnn.com' : {'head_tag' : ('', '', '') ,'text_tag':('div', 'class', ['zn-body__paragraph', 'zn-body__paragraph speakable'])},
              'www.denverpost.com' : {'head_tag' : ('div', 'class', 'article-body') ,'text_tag':('p', '', '')},
              'generic' : {'head_tag' : ('', '', '') ,'text_tag':('p', '', '')},
             }

In [3]:
# Replace with your subscription key.
subscription_key = os.environ.get('AzureAPIKey')
endpoint = 'https://westus.api.cognitive.microsoft.com/'

candidates = ['bernie sanders', 'amy klobuchar', 'joe biden', 'michael bloomberg', \
              'pete buttigieg', 'tom steyer', 'elizabeth warren']
other_search = ['', 'campaign', 'iowa', 'new hampshire', 'December', 'January', 'February']
client = NewsSearchClient(endpoint=endpoint, credentials=CognitiveServicesCredentials(subscription_key))



In [2]:
db_client = MongoClient()
db_news = db_client['news_search']
db_news_col = db_news['search_result']

In [3]:
list(db_news.list_collections())

[{'name': 'news_content',
  'type': 'collection',
  'options': {},
  'info': {'readOnly': False,
   'uuid': UUID('1f330c76-c1f5-4433-977b-daf83f3d97e7')},
  'idIndex': {'v': 2,
   'key': {'_id': 1},
   'name': '_id_',
   'ns': 'news_search.news_content'}},
 {'name': 'search_result',
  'type': 'collection',
  'options': {},
  'info': {'readOnly': False,
   'uuid': UUID('c51665c4-1137-4541-8524-807451bfc7b3')},
  'idIndex': {'v': 2,
   'key': {'_id': 1},
   'name': '_id_',
   'ns': 'news_search.search_result'}}]

In [6]:
db_news.drop_collection('search_result')
# db_news.drop_collection('news_content')

{'nIndexesWas': 1, 'ns': 'news_search.news_content', 'ok': 1.0}

In [7]:
# Run through each of the candidates and search for news on them

for candidate in candidates :
    for term in other_search :
        news_list = []
        search_term = candidate + ' ' + term
        news_result = client.news.search(query=search_term, market="en-us", count=100)

        if news_result.value:
            for result in news_result.value :
                base_url = result.url[result.url.find('www.'):result.url.find('/', result.url.find('www.'))]
                news_list.append({'name':result.name, 'url': result.url.replace(' ', ''), 'pub_date' : result.date_published, \
                            'provider' : result.provider[0].name, 'base_url' : base_url})
                
        if(len(news_list) > 0) :
            db_news_col.insert_many(news_list)

In [4]:
cursor = db_news_col.find({}, {'_id':1, 'name':1, 'url':1, 'provider' : 1, 'base_url':1, 'pub_date':1})

In [5]:
len(list(cursor))

4490

In [19]:
def scrape_news(article) :
    # Put the text in a list when we find it
    text_list = []
    
    # Make sure we have valid data in this list
    if (article['base_url'] != '') :
        if article['base_url'] in div_search.keys() :
            # Now figure out what parts of the page we should be looking for
            paper_dict = div_search[article['base_url']]
        else :
            paper_dict = div_search['generic']
            
        
        no_head_tag = False
        soup_list = []
        
        # Read the response from the web site
        try :
            response = requests.get(article['url'], auth=('user', 'pass'))
        except :
            print('Broken pipe on : ', article['url'])
            return text_list
        
        # Parse the response in Beautiful Soup
        article_soup = soup(response.text, 'html5lib')
 
        if paper_dict['head_tag'][1] != '' :
            head_search_attrs = { paper_dict['head_tag'][1] : paper_dict['head_tag'][2]}
        else :
            head_search_attrs = {}
        
        if paper_dict['text_tag'][1] != '' :
            text_search_attrs = { paper_dict['text_tag'][1] : paper_dict['text_tag'][2] }
        else :
            text_search_attrs = {}


        
        # Find the beginning of the article text
        head_tag = None
        if paper_dict['head_tag'][0] != '' :
            head_tag = article_soup.find(paper_dict['head_tag'][0], attrs = head_search_attrs)
        else :
            no_head_tag = True
            
        # Find all the descendants that contain the text of the article
        if head_tag != None :
            # print('Head Found')
            if text_search_attrs :
                soup_list = head_tag.find_all(paper_dict['text_tag'][0], attrs=text_search_attrs)
            else :
                soup_list = head_tag.find_all(paper_dict['text_tag'][0])
        
        elif (no_head_tag == True) or (head_tag == None):
            if text_search_attrs :
                soup_list = article_soup.find_all(paper_dict['text_tag'][0], attrs=text_search_attrs)
            else :
                soup_list = article_soup.find_all(paper_dict['text_tag'][0])
        
        # print(article['base_url'], head_search_attrs, text_search_attrs, len(soup_list))
                
        for paragraph in soup_list :
            text_list.append(paragraph.text)
        
        return(text_list)
        


In [20]:
cursor = db_news_col.find({}, {'_id':1, 'name':1, 'url':1, 'provider' : 1, 'base_url':1, 'pub_date':1})
db_news_content = db_news['news_content']

need_learn = []
scrape_count = 0

for article in list(cursor):
    if db_news_content.count_documents({'name' : article['name']}) == 0 : 
        article_text = scrape_news(article)
        scrape_count += 1
        if scrape_count == 100 :
            print('.')
            scrape_count = 0
            
        time.sleep(2)

        if(article_text) :
            good_article = {'name' : article['name'], 'url' : article['url'], 'pub_date' : article['pub_date'], \
                    'provider' : article['provider'], 'base_url' : article['base_url'], 'text' : article_text}

            db_news_content.insert_one(good_article)
        else :
            need_learn.append(article['base_url'])
        

.
Broken pipe on :  https://www.thenewstribune.com/news/politics-government/election/article239765308.html
Broken pipe on :  https://www.thenewstribune.com/news/politics-government/election/article240364736.html
Broken pipe on :  https://www.bellinghamherald.com/latest-news/article240318386.html
Broken pipe on :  https://www.sacbee.com/news/politics-government/capitol-alert/article239701373.html
Broken pipe on :  https://www.thenewstribune.com/news/politics-government/election/article240364736.html
.
.
Broken pipe on :  https://www.newsobserver.com/entertainment/celebrities/article239574733.html
Broken pipe on :  https://www.charlotteobserver.com/news/politics-government/national-politics/article239709818.html
Broken pipe on :  https://www.tri-cityherald.com/entertainment/celebrities/article239574733.html
Broken pipe on :  https://www.centredaily.com/entertainment/celebrities/article239574733.html
Broken pipe on :  https://www.sacbee.com/news/politics-government/national-politics/artic

KeyboardInterrupt: 

In [7]:
db_news_content = db_news['news_content']
cursor = db_news_content.find({}, {'_id':0, 'name': 1, 'text': 1})
len(list(cursor))

2059

In [22]:
set(need_learn)

{'',
 'www.bellinghamherald.com',
 'www.centredaily.com',
 'www.charlotteobserver.com',
 'www.cnn.com',
 'www.counterpunch.org',
 'www.dailydot.com',
 'www.hawaiitribune-herald.com',
 'www.latimes.com',
 'www.limaohio.com',
 'www.msn.com',
 'www.newsobserver.com',
 'www.newsweek.com',
 'www.nytimes.com',
 'www.politico.com',
 'www.powerlineblog.com',
 'www.rawstory.com',
 'www.sacbee.com',
 'www.sfexaminer.com',
 'www.staradvertiser.com',
 'www.tabletmag.com',
 'www.thenewstribune.com',
 'www.tri-cityherald.com',
 'www.washingtonexaminer.com'}

In [26]:
cursor = db_news_col.find({'base_url':'www.newsweek.com'}, {'_id':0, 'base_url':1, 'url':1})
list(cursor)

[{'url': 'https://www.newsweek.com/bernie-sanders-democratic-socialist-not-communist-heres-difference-1485478',
  'base_url': 'www.newsweek.com'},
 {'url': 'https://www.newsweek.com/bernie-sanders-joe-rogan-endorsement-podcast-twitter-2020-1483809',
  'base_url': 'www.newsweek.com'},
 {'url': 'https://www.newsweek.com/bernie-sanders-jumps-nine-points-since-december-new-national-poll-holds-commanding-lead-among-1487763',
  'base_url': 'www.newsweek.com'},
 {'url': 'https://www.newsweek.com/fight-breaks-out-bernie-sanders-rally-denver-1487701',
  'base_url': 'www.newsweek.com'},
 {'url': 'https://www.newsweek.com/bernie-sanders-leads-iowa-7-points-poll-released-eve-caucus-night-1485330',
  'base_url': 'www.newsweek.com'},
 {'url': 'https://www.newsweek.com/bernie-sanders-jumps-nine-points-since-december-new-national-poll-holds-commanding-lead-among-1487763',
  'base_url': 'www.newsweek.com'},
 {'url': 'https://www.newsweek.com/bernie-sanders-19-point-lead-nevada-caucus-frontrunner-new-po

<!DOCTYPE html>
<html lang="en-US"><head>
				<meta charset="utf-8"/><meta content="themercurynews" id="app-name" name="application-name"/><meta content="width=device-width, initial-scale=1" name="viewport"/>			<meta content="4B535F7EB2971D1FCBA5D1D3E3E292C3" name="msvalidate.01"/>
			<title>Bernie Sanders rallies California supporters to encourage early voting</title>
<script type="text/javascript">
/* <![CDATA[ */
var scriptHelpers = {"staticPath":"https:\/\/assets.digitalfirstmedia.com\/prod\/static\/"};
/* ]]> */
</script>
		<style>body{--primary:#0090d9;--secondary:#cd2707;--tertiary:#ffc518;--primary-lighter:#a0d0ed;--primary-light:#56b7f2;--primary-dark:#31a8f0;--primary-darker:#0e7bbd}:root{--primary:#006fb7;--secondary:#d70000;--tertiary:#ffc518;--primary-lighter:#47b6ff;--primary-light:#0093f5;--primary-dark:#00568f;--primary-darker:#003d66}@font-face{font-family:icomoon;src:url(data:application/vnd.ms-fontobject;base64,RCQAAKAjAAABAAIAAAAAAAAAAAAAAAAAAAABAJABAAAAAExQAAAAAAAA